# 003: Implement logistic regression

needs data loading, train/test split, training, evaluation

In [1]:
import sys

import numpy as np

sys.path.append("../")
import helpers

In [5]:
x_train, x_test, y_train, train_ids, test_ids = helpers.load_csv_data("../data/dataset", sub_sample=True)

In [98]:
#def sigmoid(t):
#    return 1 / (1 + np.exp(-t))

def sigmoid(t):
    """apply sigmoid function on t.

    Args:
        t: scalar or numpy array

    Returns:
        scalar or numpy array
    """
    # numerically stable version
    t = np.asarray(t, dtype=np.float64)
    out = np.empty_like(t)
    pos = t >= 0
    neg = ~pos
    out[pos] = 1 / (1 + np.exp(-t[pos]))
    exp_t = np.exp(t[neg])
    out[neg] = exp_t / (1 + exp_t)
    return out

def calculate_loss(y, tx, w):
    """compute the cost by negative log likelihood.

    Args:
        y:  shape=(N, 1)
        tx: shape=(N, D)
        w:  shape=(D, 1)

    Returns:
        a non-negative loss
    """
    assert y.shape[0] == tx.shape[0]
    assert tx.shape[1] == w.shape[0]

    pred = sigmoid(tx @ w)
    # clipping for stability
    eps = 1e-15
    pred = np.clip(pred, eps, 1 - eps)
    loss = (y * np.log(pred) + (1 - y) * np.log(1 - pred))
    return - 1 / pred.shape[0] * np.sum(loss).item()


def calculate_gradient(y, tx, w, sample_weights=None):
    """compute the gradient of loss.

    Args:
        y:  shape=(N, 1)
        tx: shape=(N, D)
        w:  shape=(D, 1)

    Returns:
        a vector of shape (D, 1)
    """
    assert y.shape[0] == tx.shape[0]
    assert tx.shape[1] == w.shape[0]

    pred = sigmoid(tx @ w)

    if sample_weights is None:
        sample_weights = np.ones_like(y)

    gradient = tx.T @ (sample_weights * (pred - y)) / np.sum(sample_weights)

    assert gradient.shape == w.shape
    return gradient

def learning_by_gradient_descent(y, tx, w, gamma, sample_weights=None):
    """
    Do one step of gradient descent using logistic regression. Return the loss and the updated w.

    Args:
        y:  shape=(N, 1)
        tx: shape=(N, D)
        w:  shape=(D, 1)
        gamma: float

    Returns:
        loss: scalar number
        w: shape=(D, 1)
    """
    loss = calculate_loss(y, tx, w)
    w_new = w - gamma * calculate_gradient(y, tx, w, sample_weights)
    return loss, w_new

In [124]:
def logistic_regression_gradient_descent_demo(y, x):
    # init parameters
    max_iter = 1000
    threshold = 1e-8
    gamma = 1e-1
    losses = []

    # build tx
    tx = np.c_[np.ones((y.shape[0], 1)), x]
    w = np.zeros((tx.shape[1], 1))

    pos_weight = y.shape[0] / (2 * np.sum(y))
    print(pos_weight)
    neg_weight = y.shape[0] / (2 * np.sum(1 - y))
    print(neg_weight)
    sample_weights = np.where(y == 1, pos_weight, neg_weight).reshape(-1, 1)

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_gradient_descent(y, tx, w, gamma, sample_weights)
        # log info
        if iter % 1 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
        print("L1 norm of w:", np.sum(np.abs(w)))
        print(np.mean(tx @ w))
        
        print(np.mean(np.sign(sigmoid(tx @ w))==y))
    
    
    print("loss={l}".format(l=calculate_loss(y, tx, w)))

In [123]:
# Mean imputation for each column in x_train
col_means = np.nanmean(x_train, axis=0)
inds = np.where(np.isnan(x_train))
x_train[inds] = np.take(col_means, inds[1])

In [125]:
y = (y_train.reshape(-1, 1) + 1) / 2
# kick out features with no variance
stds = np.std(x_train, axis=0)
x_train = x_train[:, stds > 0]
print(x_train.shape)
x_train = (x_train - x_train.mean(axis=0)) / x_train.std(axis=0)
logistic_regression_gradient_descent_demo(y, x_train)

(6563, 310)
5.849376114081997
0.5467344218593803
Current iteration=0, loss=0.6931471805599453
L1 norm of w: 0.9726263362548302
-9.148386587190076e-17
0.08547920158464117
Current iteration=1, loss=0.6770011951064568
L1 norm of w: 1.6731509193354426
-0.0030612120058533653
0.08547920158464117
Current iteration=2, loss=0.6724301990015914
L1 norm of w: 2.2074323426796494
-0.00811490599879052
0.08547920158464117
Current iteration=3, loss=0.6718600467805583
L1 norm of w: 2.6348719750538443
-0.01447584872689495
0.08547920158464117
Current iteration=4, loss=0.672439927479991
L1 norm of w: 2.9866311960461287
-0.021720657466610397
0.08547920158464117
Current iteration=5, loss=0.673086797517691
L1 norm of w: 3.2833702489645957
-0.02957892903914457
0.08547920158464117
Current iteration=6, loss=0.673401638717353
L1 norm of w: 3.5397335868966575
-0.03787029480937465
0.08547920158464117
Current iteration=7, loss=0.6732638898012977
L1 norm of w: 3.7637604267080533
-0.04646958445842325
0.085479201584641